In [1]:
import pandas as pd
import psycopg2
import os
from sqlalchemy import create_engine, text, MetaData, Table

### STEP 1: Load data

#### STEP 1A: Loading Existing Data from base tables (Project, Source, Types)

In [2]:
db_params = {
    'host': os.getenv('DB_HOST') or 'localhost',
    'database': os.getenv('DB_NAME') or 'tbmc_db',
    'user': os.getenv('DB_USER') or 'tbmc_db_user',
    'password': os.getenv('DB_PASSWORD') or '123456',
    'table': os.getenv('DB_TABLE') or 'tbmc_db1',
    'port': os.getenv('DB_PORT') or '5432'
}

In [3]:
def connect_to_database(db_params):
    try:
        conn = psycopg2.connect(
            host=db_params['host'],
            database=db_params['database'],
            user=db_params['user'],
            password=db_params['password']
        )
        conn.set_session(autocommit=True)
        
        engine = create_engine(f"postgresql://{db_params['user']}:{db_params['password']}@{db_params['host']}:{db_params['port']}/{db_params['database']}")
        
        if conn and engine:
            print("Connection successful!")
            return conn, engine
        else:
            print("Connection failed.")

        
    except Exception as e:
        print(f"Error connecting to database: {e}")
        return None, None

conn, engine = connect_to_database(db_params)



Connection successful!


In [4]:
def existing_table_data():
    if engine:
        query_type = "SELECT * FROM type"
        query_source = "SELECT * FROM source"
        query_items = "SELECT * FROM items"

        item_type = pd.read_sql(query_type, engine)
        item_source = pd.read_sql(query_source, engine)
        item_items = pd.read_sql(query_items, engine)


        return item_type, item_source, item_items

    else:
        print("Failed to connect to the database.")

type_table, source_table, items_table = existing_table_data()

In [14]:
item_columns = list(items_table)

#### STEP 1A: Loading New Data

In [32]:
file_path = "../data/quotation_princing_analysis/"
file_name = "Replacement of Ventilation Duct at Cell Loading Area - Rhajtek.xlsx"
file_path_name = file_path + file_name
data = pd.read_excel(file_path_name, names=item_columns)

In [33]:
data.head(15)

,description,man_r,unit,qty,unit_cost,amount,project,type,date,source
0,"mobilization/temfacil/housing, personnel trave...",NaN,lot,1,40000.0,40000.0,REPLACEMENT OF VENTILATION DUCT AT CELL LOADIN...,Mobilization/Demobilization & Housekeeping,2024-01-15,INHOUSE
1,demobilization,NaN,lot,1,25000.0,25000.0,REPLACEMENT OF VENTILATION DUCT AT CELL LOADIN...,Mobilization/Demobilization & Housekeeping,2024-01-15,INHOUSE
2,rubber gloves,NaN,pairs,300,65.0,19500.0,REPLACEMENT OF VENTILATION DUCT AT CELL LOADIN...,Safety Provisions,2024-01-15,INHOUSE
3,dust mask n95,NaN,bxs,29,187.5,5437.5,REPLACEMENT OF VENTILATION DUCT AT CELL LOADIN...,Safety Provisions,2024-01-15,INHOUSE
4,caution tape,NaN,roll,1,1500.0,1500.0,REPLACEMENT OF VENTILATION DUCT AT CELL LOADIN...,Safety Provisions,2024-01-15,INHOUSE
5,safety signages,NaN,lot,1,5000.0,5000.0,REPLACEMENT OF VENTILATION DUCT AT CELL LOADIN...,Safety Provisions,2024-01-15,INHOUSE
6,"fire blanket (size: 2m x 2m, temperatures up t...",NaN,pcs,4,5000.0,20000.0,REPLACEMENT OF VENTILATION DUCT AT CELL LOADIN...,Safety Provisions,2024-01-15,INHOUSE
7,"welding blanket size: 2m x 2m, minimum type ra...",NaN,pcs,4,5000.0,20000.0,REPLACEMENT OF VENTILATION DUCT AT CELL LOADIN...,Safety Provisions,2024-01-15,INHOUSE
8,"coffee hygiene uniform (4 shirt, 2 pants)",NaN,pairs,16,1600.0,25600.0,REPLACEMENT OF VENTILATION DUCT AT CELL LOADIN...,Safety Provisions,2024-01-15,INHOUSE
9,safety shoes,NaN,pairs,16,1200.0,19200.0,REPLACEMENT OF VENTILATION DUCT AT CELL LOADIN...,Safety Provisions,2024-01-15,INHOUSE


In [34]:
if engine:    
    # Append the DataFrame to the table named 'items' in the connected database
    data.to_sql('items', engine, if_exists='append', index=False)
    print("DataFrame appended to the database successfully!")
else:
    print("Failed to connect to the database.")


DataFrame appended to the database successfully!


### STEP 2: Final cleaning

### STEP 3: Add project name to "Project" table

#### STEP 3A: Connect to the database

#### STEP 3B: Upload data

#### STEP 3C: Close Connection

In [12]:
conn.close()

In [13]:
if conn:
    conn.close()
    print("Connection closed.")

if engine:
    engine.dispose()
    print("Engine disposed.")

Connection closed.
Engine disposed.
